In [1]:
# Import necessary libraries
import pulp
import pandas as pd
import random  # For generating more varied mock data


In [7]:

# --- Step 1: Create Mock Data and Save to CSV ---

# Number of employees, shifts, and days
num_employees = 40
num_shifts = 3  # Day, Evening, Night
num_days = 7

# Skill levels
skills = ["RN", "LPN", "CNA"]

# Create Employee Data
employee_data = []
for i in range(num_employees):
    employee_id = f"E{i+1}"
    skill = random.choice(skills)
    max_hours = random.randint(30, 48)  # Vary max hours

    # Availability:  Start by assuming everyone's available, then randomly make some unavailable.
    availability = {}
    for day in range(num_days):
        availability[day] = [1] * num_shifts  # Initially available all shifts
        for shift in range(num_shifts):
            if random.random() < 0.2:  # 20% chance of being unavailable for a shift
                availability[day][shift] = 0

    employee_data.append([employee_id, skill, max_hours, availability])

employee_df = pd.DataFrame(employee_data, columns=["employee_id", "skill", "max_hours", "availability"])
employee_df.set_index("employee_id", inplace=True)  # Set employee_id as index
employee_df.to_csv("employees.csv")


In [8]:
# Create Shift Data
shift_data = []
shift_names = ["Day", "Evening", "Night"]
for shift_name in shift_names:
    required_staff = {}
    for skill in skills:
        # More RNs and CNAs needed during the day, fewer at night, etc.
        if shift_name == "Day":
          required_staff[skill] = random.randint(1, 3)
        elif shift_name == "Evening":
          required_staff[skill] = random.randint(0, 2)
        else: # Night
          required_staff[skill] = random.randint(0, 2)

    shift_length = 8  # Assume all shifts are 8 hours
    shift_data.append([shift_name, required_staff, shift_length])

shift_df = pd.DataFrame(shift_data, columns=["shift_id", "required_staff", "shift_length"])
shift_df.set_index("shift_id", inplace=True)
shift_df.to_csv("shifts.csv")

In [9]:
# --- Step 2: Load Data from CSV ---

employee_df = pd.read_csv("employees.csv", index_col="employee_id")
# Crucial:  The availability column is a string; convert it back to a dictionary.
employee_df['availability'] = employee_df['availability'].apply(eval)
shift_df = pd.read_csv("shifts.csv", index_col="shift_id")
shift_df['required_staff'] = shift_df['required_staff'].apply(eval)

print("Employee Data:")
print(employee_df)
print("\nShift Data:")
print(shift_df)

Employee Data:
            skill  max_hours  \
employee_id                    
E1            CNA         44   
E2            LPN         39   
E3            LPN         48   
E4            CNA         46   
E5            LPN         33   
E6             RN         30   
E7            CNA         42   
E8            CNA         43   
E9            CNA         37   
E10           CNA         43   
E11           CNA         36   
E12           LPN         35   
E13           CNA         36   
E14           LPN         46   
E15           CNA         44   
E16            RN         46   
E17           CNA         40   
E18           LPN         38   
E19            RN         30   
E20           LPN         32   
E21           CNA         39   
E22           LPN         40   
E23           LPN         42   
E24           LPN         32   
E25            RN         35   
E26           CNA         35   
E27            RN         34   
E28           CNA         32   
E29           LPN        

In [10]:
# --- Step 3: Create the MILP Model ---

model = pulp.LpProblem("Nursing_Home_Scheduling", pulp.LpMinimize)

# --- Step 4: Define Decision Variables ---

days = range(num_days)
shifts = shift_df.index.tolist()  # Get shift names from DataFrame
employees = employee_df.index.tolist()

x = pulp.LpVariable.dicts("schedule",
                          [(employee, shift, day)
                           for employee in employees
                           for shift in shifts
                           for day in days],
                          cat='Binary')

# --- Step 5: Define Objective Function ---

# Minimize total shifts (simple objective for demonstration)
model += pulp.lpSum(x[employee, shift, day]
                    for employee in employees
                    for shift in shifts
                    for day in days), "Total_Shifts"


# --- Step 6: Define Constraints ---

# 1. Staffing Requirements
for day in days:
    for shift in shifts:
        for skill, required in shift_df.loc[shift, "required_staff"].items():
            model += pulp.lpSum(x[employee, shift, day]
                                for employee in employees
                                if employee_df.loc[employee, "skill"] == skill) >= required, f"Staffing_{skill}_{shift}_{day}"

# 2. One Shift Per Day
for employee in employees:
    for day in days:
        model += pulp.lpSum(x[employee, shift, day] for shift in shifts) <= 1, f"One_Shift_{employee}_{day}"

# 3. Maximum Weekly Hours
for employee in employees:
    model += pulp.lpSum(x[employee, shift, day] * shift_df.loc[shift, "shift_length"]
                        for shift in shifts
                        for day in days) <= employee_df.loc[employee, "max_hours"], f"Max_Hours_{employee}"

# 4. Employee Availability
for employee in employees:
    for day in days:
        for shift_index, shift in enumerate(shifts):
            if employee_df.loc[employee, "availability"][day][shift_index] == 0:
                model += x[employee, shift, day] == 0, f"Availability_{employee}_{shift}_{day}"

# --- Step 7: Solve the Model ---

model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jonas/Projects/MILP-test/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e41cb9bdc1bc4e0f9a3a1db8b68e23ae-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/e41cb9bdc1bc4e0f9a3a1db8b68e23ae-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 542 COLUMNS
At line 5737 RHS
At line 6275 BOUNDS
At line 7116 ENDATA
Problem MODEL has 537 rows, 840 columns and 2674 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 77 - 0.00 seconds
Cgl0002I 154 variables fixed
Cgl0004I processed model has 325 rows, 603 columns (603 integer (603 of which binary)) and 1761 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 6 integers unsatisfied sum - 1.75
Cbc0038I Pass   1: suminf.    0.25000 (2) obj. 77 iterations 54
Cbc0038I Solution found 

1

In [13]:
# --- Step 8: Print and Interpret the Results ---

print(f"Status: {pulp.LpStatus[model.status]}")

if pulp.LpStatus[model.status] == 'Optimal':
    schedule_data = []
    for day in days:
        for shift in shifts:
            for employee in employees:
                if x[employee, shift, day].varValue > 0.9:
                    schedule_data.append([day, shift, employee, employee_df.loc[employee, 'skill']])
    schedule_df = pd.DataFrame(schedule_data, columns=['Day', 'Shift', 'Employee', 'Skill'])
    day_names = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    schedule_df['Day'] = schedule_df['Day'].map(lambda d: day_names[d])
    print("\nOptimal Schedule:")
    print(schedule_df)
    # Print schedule to CSV
    schedule_df.to_csv("nursing_home_schedule.csv", index=False)
    # Print total shifts worked
    print(f"\nTotal Shifts Worked: {pulp.value(model.objective)}")
else:
    print("No optimal solution found.")
    # If infeasible, you can use PuLP's tools to investigate:
    # (This is advanced and often requires understanding the solver's output)
    # model.writeLP("nursing_home.lp")  # Write the model to a file for inspection
    # Look for conflicting constraints.

Status: Optimal

Optimal Schedule:
    Day    Shift Employee Skill
0   Mon      Day       E1   CNA
1   Mon      Day       E5   LPN
2   Mon      Day      E15   CNA
3   Mon      Day      E26   CNA
4   Mon      Day      E40    RN
..  ...      ...      ...   ...
72  Sun  Evening      E34    RN
73  Sun  Evening      E35   LPN
74  Sun    Night      E15   CNA
75  Sun    Night      E33    RN
76  Sun    Night      E37   LPN

[77 rows x 4 columns]

Total Shifts Worked: 77.0
